# STATIC STUFF 
 
THIS PART IS USED FOR THE SETUP OF THIS NOTEBOOK

Establish connection

In [1]:
from modules.athena_queries import AthenaQueries
q = AthenaQueries()

Test connection|

In [2]:
df = q.athena_query('SELECT * FROM "default"."details" limit 10;')
df

,id,lng,lat,name,county,parish,abs_price,abs_size,title,room_nr,url,partition_0,partition_1,partition_2,partition_3
0,2867991,24.726131,59.4373683,Tallinn,1,1061,850,72,Anda üürile korter Rohu Kvartalis,3,https://www.kv.ee/2867991,rent,Tallinn,2020-03-13,3
1,3151212,24.8904477999999,59.4440622,Tallinn,1,1061,550,43,UUED ÜÜRIKORTERID. ÜÜRIHIND SISALDAB KOMMUNAALE!,3,https://www.kv.ee/3151212,rent,Tallinn,2020-03-13,3
2,3215141,24.7032279,59.4002385,Tallinn,1,1061,430,61,,3,https://www.kv.ee/3215141,rent,Tallinn,2020-03-13,3
3,3220135,24.7581046599858,59.4390972854348,Tallinn,1,1061,1600,85,LIFT SÕIDAB OTSE KORTERISSE Parkimiskohta TASUTA,3,https://www.kv.ee/3220135,rent,Tallinn,2020-03-13,3
4,3205741,24.7827781,59.4282726,Tallinn,1,1061,2200,97,Suurepärase vaatega uus 3-toaline korter,3,https://www.kv.ee/3205741,rent,Tallinn,2020-03-13,3
5,3217126,24.7272107,59.4331155,Tallinn,1,1061,800,90,Möbleeritud ja avar 3-toaline maitsekas kodu K...,3,https://www.kv.ee/3217126,rent,Tallinn,2020-03-13,3
6,3204269,24.7791448,59.4391391,Tallinn,1,1061,1000,60,ESINDUSLIK KODU KADRIOURU JA KESKLINNA PIIRIL!,3,https://www.kv.ee/3204269,rent,Tallinn,2020-03-13,3
7,3204264,24.7791448,59.4391391,Tallinn,1,1061,750,47,ESINDUSLIK KODU KADRIOURU JA KESKLINNA PIIRIL!,3,https://www.kv.ee/3204264,rent,Tallinn,2020-03-13,3
8,3198659,24.7666340000001,59.427569,Tallinn,1,1061,650,80,ÜÜRILE ANDA MÖBLEERITUD KORTER!,3,https://www.kv.ee/3198659,rent,Tallinn,2020-03-13,3
9,3219697,24.7573781,59.4392467,Tallinn,1,1061,1700,70,Hubane stiilne korter,3,https://www.kv.ee/3219697,rent,Tallinn,2020-03-13,3


In [7]:
%load_ext autoreload
%autoreload

# IDEAS TO CHECK 

+ Select apartments which have occured in my dataset less than their last_occurence - first_occurence. These do occur
+ If I am able to create a grid over the map I can find for certain areas the average rent price for that area. Based on that I can declare some apartments in certain GRID areas as great catches.




## PLOTS TO DO 

0. Visualize the division of how long apartment is active.
    + rent vs sale 
    + price_bucket 
    + city 
    + room_nr
1. Visualize the locations on map
2. Visualize apartments that went fast and that went slow TOP,BOTTOM  5 percent


In [12]:
import plotly.offline as pyo 
import plotly.graph_objs as go

# APARTMENT AVERAGE WAIT DURATION BEFORE DEAL

## RENT

In [14]:

import plotly.offline as pyo 
import plotly.graph_objs as go

def generate_avg_wait_bar(wait_times,title):
    data=[]
    for room_nr in range(1,6):
        df=wait_times[room_nr]
        trace= go.Bar(x=df['city'],y=df['avg_age'],name=f"{room_nr}")
        data.append(trace)
    layout = go.Layout(title=title)
    fig = go.Figure(data=data,layout=layout)
    return fig

In [15]:
rent_wait_times = q.get_avg_wait_time(deal_type="rent")
sale_wait_times = q.get_avg_wait_time(deal_type="sale")

In [16]:
rent_plt = generate_avg_wait_bar(rent_wait_times,"Rent Average Wait")
sale_plt = generate_avg_wait_bar(sale_wait_times,"Sale Average Wait")

In [17]:
pyo.iplot(rent_plt)

In [18]:
pyo.iplot(sale_plt)

Plotting part 

## SALE

In [19]:
res={}

deal_type="sale"
for room_nr in range(1,6):
    query = f"""with t1 as(SELECT city,room_nr,AVG(days_active) as avg_age ,count(*) as cnt FROM "default"."state_view" where days_inactive > 2 and deal_type = '{deal_type}' and room_nr = {room_nr} GROUP BY 1,2)
SELECT * FROM t1 WHERE cnt > 3;"""
    df = pd.read_sql(query,conn)
    res[room_nr]=df 

NameError: name 'pd' is not defined

In [ ]:
data=[]

for room_nr in range(1,6):
    df=res[room_nr]
    trace= go.Bar(x=df['city'],y=df['avg_age'],name=f"{room_nr}")
    data.append(trace)

layout = go.Layout(title="Average duration till deal")
fig = go.Figure(data=data,layout=layout)
pyo.iplot(fig)
